<a href="https://colab.research.google.com/github/Ireonx/titanic/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



train_data = pd.read_csv('https://raw.githubusercontent.com/Ireonx/titanic/main/train.csv', error_bad_lines=False)
test_data = pd.read_csv('https://raw.githubusercontent.com/Ireonx/titanic/main/test.csv', error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
train_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [10]:
y = train_data.Survived 
X = train_data.drop(['Survived'], axis = 1)

In [12]:
import seaborn as sb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

object_columns = list(train_data.select_dtypes(include = object))
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in object_columns if X[col].nunique() < 10]
# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(object_columns)-set(low_cardinality_cols))

In [13]:
def strat_kfold_gen_skl(x, y, k):
    kf = KFold(n_splits=k, random_state= None, shuffle = True)
    solution = []
    for train_index, test_index in kf.split(x, y):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        solution.append(tuple([x_train, y_train, x_test, y_test]))
    return solution


def fit_predict(X_train, y_train, X_test, y_test, classifier):
    classifier.fit(X_train, y_train)
    return classifier.predict(X_test)

params = {  'splitter' : ['best', 'random'],
            'max_depth': range (1,13),
            'min_samples_leaf': range (1,8),
            'min_samples_split': range (2,10,2),
            'max_features' : ['auto', 'sqrt', 'log2', None],
            'max_leaf_nodes': range(2, 20)}
def get_best_estimator(estimator, dict_of_params, X, y, cv_num):
    gscv = GridSearchCV(estimator, dict_of_params, n_jobs = 4, scoring = roc_auc_score, cv = cv_num).fit(X, y)
    return gscv.best_estimator_

In [14]:
encoder = OneHotEncoder(sparse = False, handle_unknown='ignore')
oh_calls_X = pd.DataFrame(encoder.fit_transform(X[low_cardinality_cols]))
oh_calls_X.index = X.index
oh_nums_X = X.drop(object_columns, axis = 1)
oh_X = pd.concat([oh_nums_X, oh_calls_X], axis = 1)
oh_X = oh_X.fillna(oh_X.median())
oh_X = oh_X.drop("PassengerId", axis =1)
oh_X["FamilySize"] = oh_X["SibSp"] + oh_X["Parch"]
oh_X = oh_X.drop("SibSp", axis =1)
oh_X = oh_X.drop("Parch", axis =1)

In [15]:
X_np = oh_X.to_numpy(copy = True)
y_np = y.to_numpy(copy = True)

In [ ]:
plt.subplots(nrows=2, ncols=1, figsize=(15, 10))
k  = np.arange(2,6)
accuracy_test = np.zeros(4)
accuracy_train = np.zeros(4)
for ks in k:
    cur_accuracy = np.zeros(ks)
    cur_accuracy_train = np.zeros(ks)
    classifier = get_best_estimator(DecisionTreeClassifier(), params, X, y, cv_num = ks)
    print(f"Best classifier: {classifier} for ks = {k}")
    for i in range(0, ks):
        X_train, y_train, X_test, y_test = strat_kfold_gen_skl(X_np, y_np, ks+1)[i]
        predictions = fit_predict(X_train, y_train, X_test, y_test, classifier = classifier)
        cur_accuracy[i] = (roc_auc_score(y_test, predictions))
        predictions_train = fit_predict(X_train, y_train, X_train, y_train, classifier)
        cur_accuracy_train[i] = (roc_auc_score(y_train, predictions_train))
    accuracy_test[ks-1] = np.mean(cur_accuracy)
    accuracy_train[ks-1] = np.mean(cur_accuracy_train)

plt.subplot(2,1,1)
plt.plot(k, accuracy_test)
plt.ylabel("Accuracy for test data")
plt.xlabel("K")
plt.subplot(2,1,2)
plt.plot(k, accuracy_train)
plt.ylabel("Accuracy for training data")
plt.xlabel("K")
plt.show()

In [ ]:
plt.subplots(nrows=2, ncols=1, figsize=(15, 10))
k  = np.arange(2,21)
accuracy_test = np.zeros(19)
accuracy_train = np.zeros(19)
for ks in k:
    cur_accuracy = np.zeros(ks)
    cur_accuracy_train = np.zeros(ks)
    classifier = get_best_estimator(DecisionTreeClassifier(), params, X_train, y_train, cv_num = ks)
    print(f"Best classifier: {classifier} for ks = {k}")
    for i in range(0, ks):
        X_train, y_train, X_test, y_test = strat_kfold_gen_skl(X_np, y_np, ks+1)[i]
        predictions = fit_predict(X_train, y_train, X_test, y_test, classifier = classifier)
        cur_accuracy[i] = (roc_auc_score(y_test, predictions))
        predictions_train = fit_predict(X_train, y_train, X_train, y_train, classifier)
        cur_accuracy_train[i] = (roc_auc_score(y_train, predictions_train))
    accuracy_test[ks-1] = np.mean(cur_accuracy)
    accuracy_train[ks-1] = np.mean(cur_accuracy_train)

plt.subplot(2,1,1)
plt.plot(k, accuracy_test)
plt.ylabel("Accuracy for test data")
plt.xlabel("K")
plt.subplot(2,1,2)
plt.plot(k, accuracy_train)
plt.ylabel("Accuracy for training data")
plt.xlabel("K")
plt.show()

In [ ]:
calls_test = pd.DataFrame(encoder.fit_transform(test_data[low_cardinality_cols]))
test.index = test_data.index
nums_test = test_data.drop(object_columns, axis = 1)
test = pd.concat([nums_test, calls_test], axis = 1)
test = test.fillna(oh_X.median())
test = test.drop("PassengerId", axis =1)
test["FamilySize"] = test["SibSp"] + test["Parch"]
test = test.drop("SibSp", axis =1)
test = test.drop("Parch", axis =1)
test["5"] = 0

classifier = DecisionTreeClassifier(max_depth = 17, random_state = 11, max_leaf_nodes = 60, splitter = 'best')
classifier.fit(X_np, y_np)
answer = classifier.predict(test)

submission_file = pd.concat([test_data["PassengerId"], pd.DataFrame(answer, columns = ["Survived"])], axis = 1)
submission_file.to_csv('.//titanic_submission_1.csv', index = False)